# Creating the Wumpus world


In [29]:
import numpy as np

class wumpusWorld :
  
  #Constructor 
  def __init__(self,rows,cols):
    
    self.rows = rows
    self.cols = cols

    #The Actual World Grid

    self.actualWorld = np.zeros((rows,cols) , dtype=object)

    #Agents Internal Grid
    self.beleifGrid = np.full((rows,cols), 'unknown', dtype=object)

    #Safe Grid 
    self.safeGrid = np.zeros((rows,cols), dtype=bool)

    #Danger Grid
    self.dangerGrid = np.zeros((rows,cols) ,dtype=bool)

    #Visited Grid
    self.visitedGrid = np.zeros((rows,cols), dtype=bool)

    #Variables to store starting position and gold position -- Setting up the actual world 

    self.startPos = [3,0]
    self.agent_pos = [3,0]
    self.homePos = [3,0]
    self.hasGold = False
    self.goldPos = None

    #Make everything empty in the beginning
    for i in range(rows) :
      for j in range(cols) :
        self.actualWorld[i][j] = 'Empty'

    self.actualWorld[0][0] = 'pit'
    self.actualWorld[0][3] = 'pit'
    self.actualWorld[2][2] = 'wumpus'
    self.actualWorld[2][3] = 'pit'
    self.actualWorld[3][2] = 'pit'

    self.actualWorld[1][1] = 'gold'

    print("World Initalized")
    print("Actual World: ")
    print(self.actualWorld)

c = wumpusWorld(4,4)

World Initalized
Actual World: 
[['pit' 'Empty' 'Empty' 'pit']
 ['Empty' 'gold' 'Empty' 'Empty']
 ['Empty' 'Empty' 'wumpus' 'pit']
 ['Empty' 'Empty' 'pit' 'Empty']]


# Agent Work

In [30]:
class agentModel : 
  def __init__(self,world):
    self.world = world
    self.path_taken = [(3, 0)]
    
  def updateBelief(self):
    row,col = self.world.agent_pos
    self.world.visitedGrid[row][col] = True
    self.world.beleifGrid[row][col] = 'safe'
    self.world.safeGrid[row][col] = True

    directions = {
      "up" : (row + 1, col),
      "down" : (row - 1, col),
      "right" : (row, col + 1),
      "left" : (row, col - 1)
    }

    percepts = {}
    for direction, (r, c) in directions.items():
      if 0 <= r < self.world.rows and 0 <= c < self.world.cols:        
        percepts[direction] = self.world.actualWorld[r][c]
      else:      
        percepts[direction] = 'wall'
   
    for direction, (r, c) in directions.items():
      if not (0 <= r < self.world.rows and 0 <= c < self.world.cols):
        continue
    
      percept = percepts[direction]
        
      if percept == 'gold':
        self.world.beleifGrid[r][c] = 'gold'
        self.world.goldPos = (r, c)
        self.world.safeGrid[r][c] = True
            
      elif percept == 'pit':
        self.world.beleifGrid[r][c] = 'pit'
        self.world.dangerGrid[r][c] = True
        self.world.safeGrid[r][c] = False
            
      elif percept == 'wumpus':
        self.world.beleifGrid[r][c] = 'wumpus'
        self.world.dangerGrid[r][c] = True
        self.world.safeGrid[r][c] = False
            
      elif percept == 'Empty':
        self.world.beleifGrid[r][c] = 'safe'
        self.world.safeGrid[r][c] = True

  def actionDecision(self):
    row,col = self.world.agent_pos

    if self.world.goldPos and not self.world.hasGold:
      goldRow, goldCol = self.world.goldPos
      distance = abs(goldRow - row) + abs(goldCol - col)

      if distance == 0:
        return "grab"
      if distance == 1:
        return self.move_towards(goldRow, goldCol)
    
    if self.world.hasGold:
      return self.move_home()
    
    return self.explore_safe_cells()

  def move_towards(self, target_row, target_col):
    row, col = self.world.agent_pos
    
    if target_row < row:
      return 'up'
    elif target_row > row:
      return 'down'
    elif target_col < col:
      return 'left'
    elif target_col > col:
      return 'right'
    else:
      return 'grab'
    
  def explore_safe_cells(self):
    row, col = self.world.agent_pos
    
    moves = [
      ('up', row - 1, col),
      ('down', row + 1, col),
      ('left', row, col - 1),
      ('right', row, col + 1)
    ]
    
    for action, r, c in moves:
      if 0 <= r < self.world.rows and 0 <= c < self.world.cols:
        if self.world.safeGrid[r][c] and not self.world.visitedGrid[r][c]:
          return action
    
    return self.backtrack()
    
  def backtrack(self):
    if len(self.path_taken) > 1:
      self.path_taken.pop()
      prev_pos = self.path_taken[-1]
      return self.move_towards(prev_pos[0], prev_pos[1])
    
    return 'nothing'
    
  def move_home(self):
    start_row, start_col = self.world.startPos
    return self.move_towards(start_row, start_col)

  def execute_action(self, action):
    row, col = self.world.agent_pos
    
    if action == 'up' and row > 0:
      self.world.agent_pos[0] -= 1
      self.path_taken.append(tuple(self.world.agent_pos))

    elif action == 'down' and row < self.world.rows - 1:
      self.world.agent_pos[0] += 1
      self.path_taken.append(tuple(self.world.agent_pos))

    elif action == 'left' and col > 0:
      self.world.agent_pos[1] -= 1
      self.path_taken.append(tuple(self.world.agent_pos))

    elif action == 'right' and col < self.world.cols - 1:
      self.world.agent_pos[1] += 1
      self.path_taken.append(tuple(self.world.agent_pos))

    elif action == 'grab':
      r, c = self.world.agent_pos
      if self.world.actualWorld[r][c] == 'gold':
        self.world.hasGold = True

def runAgent():
  world = wumpusWorld(4,4)  
  agent = agentModel(world)

  maxCounter = 50
  counter = 0
  while counter < maxCounter:
    agent.updateBelief()
    action = agent.actionDecision()
    agent.execute_action(action)
    
    print(f"Step {counter + 1}: at {tuple(world.agent_pos)}, action={action}")

    if (world.hasGold and world.agent_pos == world.startPos):
      print(f"Done! Found gold and returned home in {counter + 1} steps")
      break

    counter += 1
  
  if counter >= maxCounter:
    print("Max steps reached")

runAgent()

World Initalized
Actual World: 
[['pit' 'Empty' 'Empty' 'pit']
 ['Empty' 'gold' 'Empty' 'Empty']
 ['Empty' 'Empty' 'wumpus' 'pit']
 ['Empty' 'Empty' 'pit' 'Empty']]
Step 1: at (2, 0), action=up
Step 2: at (1, 0), action=up
Step 3: at (1, 1), action=right
Step 4: at (1, 1), action=grab
Step 5: at (2, 1), action=down
Step 6: at (3, 1), action=down
Step 7: at (3, 0), action=left
Done! Found gold and returned home in 7 steps
